# Creating Lists of People, Time and dates

In [4]:
people=[]
time=[]
date_of=[]

# Importing Dependencies

In [5]:
from keras.models import load_model
import cv2
import numpy as np

import smtplib
from datetime import datetime
from datetime import date
import pandas as pd

# Loading the Model

In [6]:
model = load_model('model-017.model')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

# Face Capture and Mask Detection

In [7]:
person_name  = input("Enter the name of the user: ")
mask_on = 0

while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for x,y,w,h in faces:
    
        face_img=gray[y:y+h,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
        
        
        if(mask_on == 0 and label==1):
            mask_on =1
            
        
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, str("   ") + labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break     

cv2.destroyAllWindows()
source.release()

Enter the name of the user: Karthik


# Send Mail

In [8]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
today = date.today()
date_today = date.isoformat(today)

if(mask_on==1):
    sender_email = "karthiksbh1@gmail.com"
    rec_email = "karthiksbh1@gmail.com"
    
    message = "Person named " + str(person_name) + " has Been Detected without a Face Mask in your area on " + str(date_today)
    
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender_email, "dummy@1234")

    server.sendmail(sender_email, rec_email, message)

# Store in Excel

In [9]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
today = date.today()
date_today = date.isoformat(today)

people.append(person_name)

time.append(current_time)

date_of.append(date_today)

data = pd.DataFrame({'Name of the Person':people,'Date':date_of,'Time':time})

datatoexcel = pd.ExcelWriter("Dataset.xlsx",engine = 'xlsxwriter')

data.to_excel(datatoexcel,sheet_name = 'Sheet1')

datatoexcel.save()